# 7. 함수 데커레이터와 클로저
---

함수 데커레이터는 소스 코드에 있는 함수를 '표시'해서 함수의 작동을 개선할 수 있게 해준다. 강력한 기능이지만, 데커레이터를 자유자재로 사용하려먼 먼저 클로저를 알아야 한다.

- 데커레이터를 구현하기 위해 클로저를 이해해야 한다.
- 그렇기 위해서는 파이썬 3.0에 추가된 nonlocal이 필요하다.
- 다만 클래스 중심의 엄격한 객체지향 방식을 고수한다면 이 기능을 사용하지 않고도 파이썬 프로그래머로서 아무런 지장없다.
- 콜백을 이용한 효율적인 비동기 프로그래밍과 필요에 따라 함수형 스타일로 코딩하는데에도 필수적이다.
- 이 장에서는 다음과 같은 내용을 살펴본다.

  - 파이썬이 데커레이터 구문을 평가하는 방식
  - 변수가 지역 변수인지 파이썬이 판단하는 방식
  - 클로저의 존재 이유와 작동 방식
  - nonlocal로 해결할 수 있는 문제
  
- 이런 기반으로 다음과 같은 주제도 심도있게 다룰 수 있다.

  - 잘 작동하는 데커레이터 구현하기
  - 표준 라이브러리에서 제공하는 재미있는 데커레이터들
  - 매개변수화된 데커레이터 구현하기
  
## 7.1 데커레이터 기본 지식

- 데커레이터는 다른 함수를 인수로 받는 callable이다.
- 예를 들어

----
```
@decorate
def target():
    print('running target()')
```

---

위 코드는 다음 코드와 동일하게 작동한다.

---

```
def target():
    print('running target()')
    
target = decorate(target)
```

---

두 코드의 실행 결과는 원래 함수인 `target()`함수를 가리키는 것이 아니라 `decorate(target)`이 반환한 함수를 가리키게 된다.

In [ ]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

@deco
def target():
    print('running target()')
    
target()

In [ ]:
target

엄밀히 말해서 데커레이터는 synactic sugar일 뿐이지만, 런타임에 프로그램 행위를 변경하는 메타프로그래밍의 경우 상당히 편리하다.

## 7.2 파이썬이 데커레이터를 실행하는 시점

파이썬이 모듈을 로딩하는 시점, 즉 import 할 때 실행 된다. 

In [4]:
registry = []

def register(func):
    print('running register({})'.format(func))
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
    
@register
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')
    
def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()
    
if __name__ == '__main__':
    main()

running register(<function f1 at 0x1045a9f28>)
running register(<function f2 at 0x1045a9ea0>)
running main()
registry -> [<function f1 at 0x1045a9f28>, <function f2 at 0x1045a9ea0>]
running f1()
running f2()
running f3()


위의 모듈을 임포트하면 e.g.) `import registration` 데코레이트된 함수 두 개는 바로 실행된다.

- 데커레이터 함수가 데커레이트되는 함수와 같은 모듈에 정의되어 있다. 일반적으로 실제 코드에서는 데커레이터를 정의하는 모듈과 데커레이터를 적용하는 모듈을 분리해서 구현한다.
- `register()` 데커레이터가 인수로 전달된 함수와 동일한 함수를 반환한다. 실제 코드에서 대부분의 데커레이터는 내부 함수를 정의해서 반환한다.

예제에서의 `register()` 데커레이터가 데커레이트된 함수를 그대로 반환하지만, 이 기법이 쓸모 없는 것은 아니다. django에서도 많이 사용된다. 이건 장고 써보면서 알아보시길 ㅋㅋㅋ

## 7.3 데커레이터로 개선한 전략 패턴

In [5]:
promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity(order):
    '''충성도 포인트가 1000점 이상인 고객에게 전체 5% 할인 적용'''
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

'''
그 외에 여러 프로모션 함수를 데커레이트한다.
'''

def best_promo(order):
    return max(promo(order) for promo in promos)

## 7.4 변수 범위 규칙

함수 매개변수로 정의된 지역 변수와 함수 내부에 정의되지 않은 변수를 테스트해보자

In [6]:
def f1(a):
    print(a)
    print(b)
    
f1(3)

3


NameError: name 'b' is not defined

In [7]:
b = 6
f1(3)

3
6


In [8]:
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9
    
f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

필자님께서 위의 결과를 보고 놀라셨다 하더라. 태초에 전역변수 b가 있었고 print(b) 다음에 지역변수 b가 있으니 어찌 6이 출력되지 않는고 예외를 뿜는 말이더냐?

파이썬께서 가라사대, 파이썬께서 함수를 컴파일을 하여 함수 내부의 b가 함수안에서 할당이 되니 어리숙한 중생들이 b를 전역에 아무리 삽입한다 하여도, 이는 함수가 고려하지 않는다는 말이다.

함수 내부의 변수는 지역변수로 판단됨을 기억하도록 하여라.

In [9]:
b = 6
def f3(a):
    global b
    print(a)
    print(b)
    b = 9
    
f3(3)

3
6


In [10]:
b

9

In [11]:
f3(3)

3
9


## 7.5 클로저

- 익명함수와 혼동하지 마라.
- 실제로 클로저는 함수 본체에서 정의하지 않고 참조하는 비전역 변수를 포함한 확장 범위를 가진 함수다.
- 함수 본체 외부에 정의된 비전역 변수에 접근할 수 있다는 것이 중요햐다.
- 나도 쓰면서 무슨 말인지 잘 모르겠으니까 예제를 한번 보는게 어떤가?

`avg()` 함수가 점차 증가하는 일련의 값의 평균을 계산한다고 가정해보자. 매일 새로운 가격이 추가되고 추가된 것을 반영하여 지금까지 모든 가격의 평균을 구하는 것이다. 이동평균

In [12]:
class Averager:
    
    def __init__(self):
        self.series = []
        
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)
    
avg = Averager()
avg(10)

10.0

In [13]:
avg(11)

10.5

In [14]:
avg(12)

11.0

고위 함수 `make_averager()`를 이용해서 구현해보자.

In [15]:
def make_averager():
    series = []
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    
    return averager

avg = make_averager()
avg(10)

10.0

In [16]:
avg(11)

10.5

In [17]:
avg(12)

11.0

Averager 클래스의 작동방식은 알겠는데 `make_averager()`는 어떻게 되는 걸까? series라는 지역변수는 averager를 반환한 후에 사라지는 지역변수인데 어떻게 데이터를 저장하고 있는가? 여기서 series는 자유변수free variable라고 한다. 자유 변수라는 말은 지역 범위에 바인딩되어 있지 않은 변수를 의미한다.

![clo](./07-1.png)

In [18]:
avg.__code__.co_varnames

('new_value', 'total')

In [19]:
avg.__code__.co_freevars

('series',)

series에 대한 바인딩은 반환된 `avg()` 함수의 `__closure__` 속성에 저장된다.

In [20]:
avg.__closure__

(<cell at 0x104587b88: list object at 0x1044d6948>,)

In [21]:
avg.__closure__[0].cell_contents

[10, 11, 12]

클로저는 함수를 정의할 때 존재하던 자유 변수에 대한 바인딩을 유지하는 함수이다. 따라서 함수를 정의하는 범위가 사라진 후에 함수를 호출해도 자유 변수에 접근할 수 있다.

## 7.6 nonlocal 선언

앞에서 `구현한 make_averager()`는 그리 효율적이지 않다. 이전 합계와 항목수가 저장되어 있으면 더 효육적으로 구현할 수 있다. 아래 코드를 보자. 잘못된 코드다. 찾아봐라

In [22]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        count += 1
        total += new_value
        return total / count
    
    return averager

In [23]:
avg = make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

왜 위와 같은 문제가 생겼지? hint. 가변형과 불변형 변수, 지역변수

어찌됐든 위와 같은 문제를 해결하기 위해 파이썬3에서 nonlocal 선언이 소개되었다. 변수를 nonlocal로 선언하면 함수 안에서 변수에 새로운 값을 할당해도 그 변수는 자유 변수임을 나타낸다.

In [24]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    
    return averager

avg = make_averager()
avg(10)

10.0

In [25]:
avg(11)

10.5

In [26]:
avg(12)

11.0

## 7.7 간단한 데커레이터 구현하기

다음 예제는 데커레이트된 함수를 호출할 때마다 시간을 측정해서 실행에 소요된 시간, 전달된 인수, 반환값을 출력하는 데커레이터다.

In [27]:
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [28]:
@clock
def snooze(seconds):
    time.sleep(seconds)
    
@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__ == '__main__':
    print('*' * 40, 'Calling snooze(.123)')
    snooze(.123)
    print('*' * 40, 'Calling factorial(20)')
    print('20! =', factorial(20))

**************************************** Calling snooze(.123)
[0.12805914s] snooze(0.123) -> None
**************************************** Calling factorial(20)
[0.00000226s] factorial(1) -> 1
[0.00001924s] factorial(2) -> 2
[0.00003021s] factorial(3) -> 6
[0.00004074s] factorial(4) -> 24
[0.00005767s] factorial(5) -> 120
[0.00006614s] factorial(6) -> 720
[0.00007997s] factorial(7) -> 5040
[0.00009374s] factorial(8) -> 40320
[0.00010846s] factorial(9) -> 362880
[0.00012216s] factorial(10) -> 3628800
[0.00013654s] factorial(11) -> 39916800
[0.00015120s] factorial(12) -> 479001600
[0.00016597s] factorial(13) -> 6227020800
[0.00018046s] factorial(14) -> 87178291200
[0.00019416s] factorial(15) -> 1307674368000
[0.00020756s] factorial(16) -> 20922789888000
[0.00022339s] factorial(17) -> 355687428096000
[0.00023871s] factorial(18) -> 6402373705728000
[0.00024978s] factorial(19) -> 121645100408832000
[0.00026076s] factorial(20) -> 2432902008176640000
20! = 2432902008176640000


어떻게 동작하는 것인가??

clock()은 factorial()함수를 func 인수로 받는다. 그 후에 clocked() 함수를 만들어서 반환하는데, 파이썬 인터프리터가 내부적으로 clocked()를 factorial에 할당했다. 

In [29]:
factorial.__name__

'clocked'

그러므로 factorial은 실제로 clocked() 함수를 참조한다. 이제부터 factorial(n)을 호출하면 clocked(n)이 실행된다. 본질적으로 clocked() 함수는 다음과 같은 연산을 수행한다.

1. 초기 시각 t0을 기록한다.
2. 원래의 factorial() 함수를 호출하고 결과를 저장한다.
3. 흘러간 시간을 계산한다.
4. 수집한 데이터를 포맷하고 출력한다.
5. 2번째 단계에서 저장한 결과를 반환한다.

위의 예제는 키워드 인수를 지원하지 않고, 데커레이트된 함수의 `__name__`과 `__doc__` 속성을 가린다.

## 7.8 표준 라이브러리에서 제공하는 데커레이터

파이썬에서는 메서드를 데커레이트하기 위해 property(), classmethod(), staticmethod() 등 총 3개의 내장 함수를 제공한다. 이는 나중에 설명한다.

그리고 자주 볼 수 있는 데커레이터 중 functools.wraps()가 있다. 이 함수는 제대로 작동하는 데커레이터를 만들기 위한 헬퍼이다. 표준 라이브러리가 제공하는 데커레이터들 중 lru_cache()와 파이썬 3.4에 추가된 완전히 새로운 singledispatch()가 가장 흥미롭다. 이 두 데커레이터는 functools 모듈에 정의되어 있다.

### 7.8.1 `functools.lru_cache()`를 이용한 memoization

- 이거 쓸모가 많다. 
- 메모이제이션은 이전에 실행한 값비싼 함수의 결과를 저장함으로써 이전에 사용된 인수에 대해 다시 계산할 필요가 없게 해준다.
- LRU는 Least Recently Used의 약자다.

n번째 피보나치 수열을 생성하기 위해 아주 느리게 실행되는 재귀 함수에서 lru_cache() 데커레이터가 진가를 발휘한다.

In [30]:
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

if __name__ == '__main__':
    print(fibonacci(6))

[0.00000063s] fibonacci(0) -> 0
[0.00000050s] fibonacci(1) -> 1
[0.00005480s] fibonacci(2) -> 1
[0.00000038s] fibonacci(1) -> 1
[0.00000079s] fibonacci(0) -> 0
[0.00000038s] fibonacci(1) -> 1
[0.00001647s] fibonacci(2) -> 1
[0.00003207s] fibonacci(3) -> 2
[0.00010275s] fibonacci(4) -> 3
[0.00000037s] fibonacci(1) -> 1
[0.00000038s] fibonacci(0) -> 0
[0.00000039s] fibonacci(1) -> 1
[0.00001510s] fibonacci(2) -> 1
[0.00003069s] fibonacci(3) -> 2
[0.00000039s] fibonacci(0) -> 0
[0.00000038s] fibonacci(1) -> 1
[0.00001519s] fibonacci(2) -> 1
[0.00000038s] fibonacci(1) -> 1
[0.00000052s] fibonacci(0) -> 0
[0.00000033s] fibonacci(1) -> 1
[0.00001602s] fibonacci(2) -> 1
[0.00003448s] fibonacci(3) -> 2
[0.00007271s] fibonacci(4) -> 3
[0.00011717s] fibonacci(5) -> 5
[0.00023563s] fibonacci(6) -> 8
8


fibonacci(1)이 8번, fibonacci(2)가 5번 호출 되는 등 계산 낭비가 엄청나다.

In [31]:
import functools

@functools.lru_cache()
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

if __name__ == '__main__':
    print(fibonacci(6))

[0.00000065s] fibonacci(0) -> 0
[0.00000054s] fibonacci(1) -> 1
[0.00005514s] fibonacci(2) -> 1
[0.00000093s] fibonacci(3) -> 2
[0.00007357s] fibonacci(4) -> 3
[0.00000067s] fibonacci(5) -> 5
[0.00009074s] fibonacci(6) -> 8
8


lru_cache()는 두 개의 선택적 인수를 이용해서 설정할 수 있다는 점에 주의하라. lru_cache()의 전체 signature는 다음과 같다.

`functools.lru_cache(maxsize=128, typed=False)`

- maxsize 인수는 얼마나 많은 호출을 저장할지 결정한다. 캐시가 가득차면 가장 오래된 결과를 버리고 공간을 확보한다.
- 최적의 성능을 내기 위해 maxsize는 2의 제곱이 되어야 한다. 
- typed 인수가 true로 설정되는 경우 인수의 자료형이 다르면 결과를 따로 저장한다.
- 예를 들어 일반적으로 1과 1.0은 동일하다고 가정하지만 실수형 인수와 정수형 인수를 구분해야 하는 경우가 있을 것이다.
- 어쨋든 `lru_cache()`가 결과를 저장하기 위해 딕셔너리를 사용하고, 호출할 때 사용한 위치 인수와 키워드 인수를 키로 사용하므로, 데커레이트된 함수가 받는 인수는 모두 해시 가능해야 한다.

### 7.8.2 단일 디스패치를 이용한 범수 함수 Generic functions with single dispatch

웹 애플리케이션을 디버깅하는 도구를 만들고 있다고 가정하자. 파이썬 객체의 자료형마다 HTML 코드를 생성하고자 한다.

In [ ]:
import html

def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

이 코드는 모든 파이썬 자료형을 잘 처리하지만 일부 자료형에 대해 고유한 코드를 생성하도록 이 코드를 확장하려 한다.

- str: 개행 문자를 `<br>\n`으로 대체하고 `<pre>`대신 `<p>` 태그를 사용한다.
- int: 숫자를 10진수와 16진수로 보여준다.
- list: 각 항목을 자료형에 따라 포맷한 HTML 리스트를 출력한다.

우리가 원하는 결과는 아래와 같음

In [32]:
r"""
htmlize(): generic function example

# BEGIN HTMLIZE_DEMO

>>> htmlize({1, 2, 3})  # <1>
'<pre>{1, 2, 3}</pre>'
>>> htmlize(abs)
'<pre>&lt;built-in function abs&gt;</pre>'
>>> htmlize('Heimlich & Co.\n- a game')  # <2>
'<p>Heimlich &amp; Co.<br>\n- a game</p>'
>>> htmlize(42)  # <3>
'<pre>42 (0x2a)</pre>'
>>> print(htmlize(['alpha', 66, {3, 2, 1}]))  # <4>
<ul>
<li><p>alpha</p></li>
<li><pre>66 (0x42)</pre></li>
<li><pre>{1, 2, 3}</pre></li>
</ul>

# END HTMLIZE_DEMO
"""

# 파이썬은 메서드나 함수의 오버로딩을 지원하지 않으므로, 서로 다르게 처리하고자 하는 자료형별로 서로 다른 시그너처를 가진 htmlize()를 만들 수 없다.
# 이때 파이썬에서는 일반적으로 htmlize()를 디스패치 함수로 변경하고, 
# 일련의 if/elif/elif 문을 이용해 htmlize_str(), htmlize_int() 등의 특화된 함수를 호출한다.
# 이렇게 되면 코드 겁나 커지고 확장하기 어렵고.. 하여튼 별로
# 파이썬 3.4에서 새로 소개된 functools.singledispatch() 데커레이터는 각 모듈이 전체 해결책에 기여할 수 있게 해준다.
# 또한 여러분(날 말하는건가?)이 편집할 수 없는 클래스에 대해서도 특화된 함수를 쉽게 제공할 수 있게 해준다.
# 일반함수를 @singledispatch로 데커레이트하면, 이 함수는 범용함수(generic function)가 된다. 
# 즉, 일련의 함수가 첫 번째 인수의 자료형에 따라 서로 다른 방식으로 연산을 수행하게 된다.

from functools import singledispatch
from collections import abc
import numbers
import html

@singledispatch  # <1>
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

@htmlize.register(str)  # <2>
def _(text):            # <3>
    content = html.escape(text).replace('\n', '<br>\n')
    return '<p>{0}</p>'.format(content)

@htmlize.register(numbers.Integral)  # <4>
def _(n):
    return '<pre>{0} (0x{0:x})</pre>'.format(n)

@htmlize.register(tuple)  # <5>
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

In [33]:
htmlize({1, 2, 3})

'<pre>{1, 2, 3}</pre>'

In [34]:
htmlize(abs)

'<pre>&lt;built-in function abs&gt;</pre>'

In [35]:
htmlize('Heimlich & Co.\n- a game')

'<p>Heimlich &amp; Co.<br>\n- a game</p>'

In [36]:
htmlize(42)

'<pre>42 (0x2a)</pre>'

In [37]:
print(htmlize(['alpha', 66, {3, 2, 1}]))

<ul>
<li><p>alpha</p></li>
<li><pre>66 (0x42)</pre></li>
<li><pre>{1, 2, 3}</pre></li>
</ul>


## 7.9 누적된 데커레이터

데커레이터가 여러개 누적되어 사용된 경우.

하나의 함수 f()에 두 테커레이터 @d1과 @d2를 차례대로 적용하면 결과는 f = d1(d2(f))와 같다.

```
@d1
@d2
def f():
    print('f')



def f():
    print('f')
    
f = d1(d2(f))
```


## 7.10 매개변수화된 데커레이터

- 소스코드에서 데커레이터를 파싱할 때 파이썬을 데커레이트된 함수를 가져와서 데커레이터 함수의 첫 번째 인수로 넘겨준다.
- 그렇다면 어떻게 다른 인수를 받는 데커레이터를 만들 수 있을까?
- 인수를 받아 데커레이터를 반환하는 데커레이터 팩토리를 만들고 나서, 데커레이트될 함수에 데커레이터 팩토리를 적용하면 된다.
- 글을 쓰면서도 뭔말인지 모르겠다. 아까 예제로 설명을 시도해본다. 이래도 이해 안되면 당신의 두뇌를 탓해라.

In [38]:
registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
    
print('running main()')
print('registry ->', registry)
f1()

running register(<function f1 at 0x1045cb378>)
running main()
registry -> [<function f1 at 0x1045cb378>]
running f1()


### 7.10.1 매개변수화된 등록 데커레이터

- register()가 등록하는 함수를 활성화 또는 비활성화하기 쉽게 만들기 위해, 선택적 인수 active를 만든다.
- active가 False면 데커레이트된 함수를 등록해제한다.
- 새로 만든 register()함수는 개념적으로는 테커레이터가 아니라 데커레이터 팩토리다.
- 호출되면 대상 함수에 적용한 실제 데커레이터를 반환하기 때문이다.

In [39]:
# BEGIN REGISTRATION_PARAM

registry = set()  # <1>

def register(active=True):  # <2>
    def decorate(func):  # <3>
        print('running register(active=%s)->decorate(%s)'
              % (active, func))
        if active:   # <4>
            registry.add(func)
        else:
            registry.discard(func)  # <5>

        return func  # <6>
    return decorate  # <7>

@register(active=False)  # <8>
def f1():
    print('running f1()')

@register()  # <9>
def f2():
    print('running f2()')

def f3():
    print('running f3()')

# END REGISTRATION_PARAM

running register(active=False)->decorate(<function f1 at 0x104633bf8>)
running register(active=True)->decorate(<function f2 at 0x1045a9f28>)
